![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Introduction to Redis Python

This notebook introduces [Redis](https://redis.io) and the standard Python client, [redis-py](https://redis-py.readthedocs.io/en/stable/), for interacting with the database. We will explore the basics of Redis setup, data structures, and capabilities like vector search!

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/RAG/00_intro_redispy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Environment Setup

### Pull Github Materials
Because you are likely running this notebook in **Google Colab**, we need to first
pull the necessary dataset and materials directly from GitHub. The following commands grab the material, move to root for colab, and clean up unneeded files.

**If you are running this notebook locally**, FYI you may not need to perform this
step at all.

In [1]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/RAG/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 212 (delta 19), reused 34 (delta 10), pack-reused 154
Receiving objects: 100% (212/212), 9.77 MiB | 10.24 MiB/s, done.
Resolving deltas: 100% (58/58), done.
mv: rename temp_repo/python-recipes/RAG/resources to ./resources: Directory not empty


### Install Python Dependencies

In [1]:
# NBVAL_SKIP
!pip install -q redis pandas unstructured[pdf] sentence-transformers langchain langchain-community

You should consider upgrading via the '/Users/robert.shelton/.pyenv/versions/3.9.12/bin/python3.9 -m pip install --upgrade pip' command.


### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.**

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Hello World Redis

Now let's connect to the Redis db and get a basic feel for the most common
commands and data structures.

In [3]:
import redis
import json
import numpy as np

from time import sleep

# Connect with the Redis Python Client
client = redis.Redis.from_url(REDIS_URL)

client.ping()

True

In [4]:
client.dbsize() # should be empty

0

Redis, at it's core, is a simple key/value store. It supports a number of interesting
and flexible data structures that can solve a variatey of business and operational
problems.

### Strings

The basic string data type can be accessed using set/get methods. You can also place a
TTL policy (expiration) on any key in Redis.

In [5]:
client.set("hello", "world")

True

In [6]:
client.get("hello")

b'world'

In [7]:
client.delete("hello")

1

In [8]:
client.set("hello", "world")
client.expire("hello", time=3)

sleep(4)

# should be EMPTY
client.get("hello")

### Hashes
Hashes are collections of key/value pairs that are grouped together. It gets
serialized as a string in Redis, but can hold a variety of data in each field.

You can think of a Hash as a one-level deep Python dictionary.


In [9]:
obj = {
    "user": "john",
    "age": 45,
    "job": "dentist",
    "bio": "long form text of john's bio",
    "user_embedding": np.array([0.3, 0.4, -0.8], dtype=np.float32).tobytes() # cast vectors to bytes string
}

In [10]:
client.hset("user:john", mapping=obj)

5

In [11]:
client.hgetall("user:john")

{b'user': b'john',
 b'age': b'45',
 b'job': b'dentist',
 b'bio': b"long form text of john's bio",
 b'user_embedding': b'\x9a\x99\x99>\xcd\xcc\xcc>\xcd\xccL\xbf'}

In [12]:
client.delete("user:john")

1

### JSON
With the JSON capabilitie enabled, Redis can be a drop-in replacement for MongoDB
or other slower document databases. You can store nested and structured JSON data
directly in Redis.

In [13]:
# set a JSON obj
obj = {
    "user": "john",
    "metadata": {
        "age": 45,
        "job": "dentist",
    },
    "user_embedding": [0.3, 0.4, -0.8]
}

client.json().set("user:john", "$", obj)

True

In [14]:
# get user JSON obj
client.json().get("user:john")

{'user': 'john',
 'metadata': {'age': 45, 'job': 'dentist'},
 'user_embedding': [0.3, 0.4, -0.8]}

In [15]:
# grab array length for embedding field
client.json().arrlen("user:john", "$.user_embedding")

[3]

In [16]:
# grab obj keys
client.json().objkeys("user:john", "$")

[[b'user', b'metadata', b'user_embedding']]

In [17]:
# delete user JSON
client.delete("user:john")

1

### Lists
Lists store sequences of information... potentially list of messages in an LLM
converstion flow, or really any list of items in a queue.

In [18]:
# add items to a list
client.rpush("messages:john", *[
    json.dumps({"role": "user", "content": "Hello what can you do for me?"}),
    json.dumps({"role": "assistant", "content": "Hi, I am a helpful virtual assistant."})
])

2

In [19]:
# list all items in the list using indices
[json.loads(msg) for msg in client.lrange("messages:john", 0, -1)]

[{'role': 'user', 'content': 'Hello what can you do for me?'},
 {'role': 'assistant', 'content': 'Hi, I am a helpful virtual assistant.'}]

In [20]:
# count items in the list
client.llen("messages:john")

2

In [21]:
# pop the first item from the list and push to another list
client.rpoplpush("messages:john", "read_messages:john")

b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}'

In [22]:
client.lrange("read_messages:john", 0, -1)

[b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}']

In [23]:
# list cleanup
client.delete("messages:john", "read_messages:john")

2

### Pipelines
All Redis commands can be pipelined to gain some round trip latency improvements.

In [24]:
with client.pipeline(transaction=False) as pipe:
    for i in range(50):
        pipe.json().set(f"user:{i}", "$", obj)
    # execute batch
    pipe.execute()

In [25]:
client.dbsize()

50

In [26]:
# clean up!
client.flushall()

True

## Intro to Vector Search in Redis

Now that we have the basics down, we will dive into the fundamentals of vector
search in Redis using the base Python client.

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs from a folder
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

Listing available documents ... ['resources/nke-10k-2023.pdf', 'resources/amzn-10k-2023.pdf', 'resources/jnj-10k-2023.pdf', 'resources/aapl-10k-2023.pdf', 'resources/nvd-10k-2023.pdf', 'resources/msft-10k-2023.pdf']


In [28]:
# fetch the Nike PDF
doc = [doc for doc in docs if "nke" in doc][0]

# set up the file loader/extractor and text splitter to create chunks
loader = UnstructuredFileLoader(doc, mode="single", strategy="fast")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=0)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Done preprocessing. Created 180 chunks of the original pdf resources/nke-10k-2023.pdf


In [29]:
# Take a look at content from a chunk
print(chunks[25].page_content)

NIKE is a consumer products company and the relative popularity of various sports and fitness activities and changing design trends affect the demand for our products, services and experiences. The athletic footwear, apparel and equipment industry is highly competitive both in the United States and worldwide. We compete internationally with a significant number of athletic and leisure footwear companies, athletic and leisure apparel companies, sports equipment companies, private labels and large companies that have diversified lines of athletic and leisure footwear, apparel and equipment. We also compete with other companies for the production capacity of contract manufacturers that produce our products. In addition, we and our contract manufacturers compete with other companies and industries for raw materials used in our products. Our NIKE Direct operations, both through our digital commerce operations and retail stores, also compete with multi-brand retailers, which sell our product

### Text embedding generation with SentenceTransformers

In [30]:
from sentence_transformers import SentenceTransformer

# load model - may take a minute or two to download the first time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/Users/robert.shelton/.pyenv/versions/3.9.12/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/robert.shelton/.pyenv/versions/3.9.12/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
%%time

# create embeddings
chunk_embeddings = model.encode([chunk.page_content for chunk in chunks])
len(chunk_embeddings) == len(chunks)

CPU times: user 694 ms, sys: 255 ms, total: 949 ms
Wall time: 2.57 s


True

### Set up some helper functions

Helper functions to encode the single query vector and display redis search results

In [32]:
import pandas as pd


def encode_one(input):
    return model.encode(input).astype(np.float32).tobytes()


def table_view(res):
    if res.total == 0:
        print("No documents found.")
        return None
    
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    return res_df


### Define a schema and create an index
Below we connect to Redis and create an index for vector search that contains a single text field and vector field.

In [33]:
from redis.commands.search.field import TagField, TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


index_name = "redispy"
key_prefix = f"doc:{index_name}"


def create_index(index_type: str = "FLAT"):       # Creates a FLAT index by default
    try:
        # check to see if index exists
        client.ft(index_name).info()
        print("Index already exists!")
    except:
        # define schema
        schema = (
            TagField("doc_id"),                    # Tag Field - synthetic ID
            TextField("content"),                  # Text Field
            VectorField("chunk_vector",            # Vector Field
                index_type, {                      # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",
                    "DIM": 384,                    # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.HASH)

        # create Index
        client.ft(index_name).create_index(fields=schema, definition=definition)

In [34]:
# Create the index
create_index()

In [35]:
# Check the info related to the newly created index
client.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'doc_id',
   b'attribute',
   b'doc_id',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b','],
  [b'identifier',
   b'content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   384,
   b'distance_metric',
   b'COSINE']],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.00818634033203125',
 'total_inverted_index_blocks': '0',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': 'nan',
 'bytes_per_record_avg':

### Process and load data using Redis
Below we use a Redis pipeline (not a transaction) to batch send writes to Redis. This method helps with throughput significantly. The batch_size param can be customized and benchmarked on your hardware and with your data. We typically recommend starting small (100-200) and increasing as needed.

In [36]:
# load expects an iterable of dictionaries

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        data = {
            'doc_id': f"{i}",
            'content': chunk.page_content,
            # For HASH -- must convert embeddings to bytes
            'chunk_vector': np.array(chunk_embeddings[i]).astype(np.float32).tobytes()
        }
        pipe.hset(f"{key_prefix}:{i}", mapping=data)
        # execute in "mini batches"
        if i % batch_size == 0:
            res = pipe.execute()

    # cleanup final batch execution
    res = pipe.execute()

In [37]:
# check the data size in Redis
len(chunks) == client.dbsize()

True

In [38]:
client.hgetall(f"{key_prefix}:0")

{b'chunk_vector': b'\xc2\xbfZ\xbde^\xef\xbcJ\xda\x83\xbb<;l=\x0f\xdf\xaf;/\x8a\x8d=\xd1X\r\xbc\x838\x00<\xbd\xd9e\xbd\xa2\x91\xf8<\xd2\xcb\x82=\xbd\x80\x1c\xbc\xb9\x14\xbf\xbcA\xdc\x95\xbbN=\xea\xbcu\xb6\x07=+5\x94<\xd6\xf8\x06=\xc64+=*\x87\x90=\xdc2\xff<\x82:\xff\xbc\xac\x15\x9e\xbd\xd1l\xa6\xbdk\xe9\x8d=p\r\xb8;\xb8\x7fr\xbd\xd2}\x08\xbdu\xe1\x93\xbd\xeb\xc8\xae\xbc\xd8\x00e\xbd\n\r+\xbd\xb4Z\x87;p\xad\xc4<V\x81\xed=\x0f\r\x9a\xbd\xafE\xdf=\xe4\xc8\xcb\xbc\xe6\x86\xde=\xf0\x1dn\xbb\x8cwo;<\x83\x8f;`\xadV=([\t=\x00\x1ee<X^\x98:\xb0\x86q\xbd\x18\xbc\xb2;\xf8\x00\xb2\xbd[\xfd\x8d=\x19\xa7\x0e=\xc7%\xa7=\x1cz\x11<\xbd\x1d\xac=\xc3\x80\xb1<,\x18B<)\x92\xae\xb9\xc4d\x97< \x9a\xd6\xbc\xb4\xfd\x03=\x18\x92\xb0\xbc|\x8c(\xbd\xd1\x02H\xbd"\xec\x1d<9\xd57=\xc8H\x8d<yC3=w \x99\xbd\x88\xf7\xe8;\xab\xfd\x89\xbd\xe1l\xe9\xbdP\x84\x7f\xbd\xeaM\xbf\xbd\xae\xa0\x98\xbc\x89\x99\xbc\xbc\x84p"\xbd\xe8\xac\x1d=}:\n=s\x19\xb2:\xb4\x93\xe1\xbdu\xe9|\xbdJs\xc0\xbb\xe1\xa2\x86<\xa0\x99O\xbc\xc5\xe0\xef<\nI$\x

### Query the database
Now we can use the Redis search index to perform vector similarity search operations.

The code below takes a user input, converts to embeddings, and fetches the top 2 most semantically similar chunks from Redis.

In [39]:
# Grab user input
_input = "Nike profit margins and company performance"

query = (
    Query("*=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,content
0,doc:redispy:85,0.321346998215,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."
1,doc:redispy:84,0.328033804893,As discussed in Note 15 — Operating Segments a...


In [40]:
# Example of sorting by a field other than vector_distance
query = (
    Query("*=>[KNN 4 @chunk_vector $vector as vector_distance]")
     .sort_by("doc_id")
     .return_fields("doc_id", "content", "vector_distance")
     .paging(0, 4)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,doc_id,content
0,doc:redispy:118,0.358749747276,118,"NIKE, INC. CONSOLIDATED STATEMENTS OF INCOME\n..."
1,doc:redispy:158,0.360825300217,158,Tax (expense) benefit Gain (loss) net of tax\n...
2,doc:redispy:84,0.328033804893,84,As discussed in Note 15 — Operating Segments a...
3,doc:redispy:85,0.321346998215,85,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."


### Range Queries
Range queries allow you to set a pre defined "threshold" for which we want to return documents

In [41]:
query = (
    Query("@chunk_vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

,id,vector_distance,content
0,doc:redispy:85,0.321346998215,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."
1,doc:redispy:84,0.328033804893,As discussed in Note 15 — Operating Segments a...
2,doc:redispy:118,0.358749747276,"NIKE, INC. CONSOLIDATED STATEMENTS OF INCOME\n..."
3,doc:redispy:158,0.360825300217,Tax (expense) benefit Gain (loss) net of tax\n...
4,doc:redispy:81,0.363789737225,"NIKE Brand revenues, which represented over 90..."
5,doc:redispy:82,0.370122373104,"Lower margin in our NIKE Direct business, driv..."
6,doc:redispy:80,0.386211931705,"4,780 (508)\n\n7 % -80 %\n\nTOTAL NIKE BRAND W..."
7,doc:redispy:86,0.393618881702,Apparel revenues increased 9% on a currency-ne...
8,doc:redispy:89,0.394767940044,"3 % -4 %\n\n13 % 4 %\n\n1,494 190\n\n8 % 23 %\..."
9,doc:redispy:159,0.395310282707,ASIA PACIFIC & LATIN AMERICA\n\n(1)\n\nGLOBAL ...


### Add filter statements
Redis queries can contain both vector search and traditional filters (numeric, tags, text, geo) in one single command.

In [42]:
# filter for docs that contain "profit" in the content field and do KNN vector search
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

,id,vector_distance,content
0,doc:redispy:81,0.363789737225,"NIKE Brand revenues, which represented over 90..."
1,doc:redispy:79,0.616192996502,"(Dollars in millions, except per share data)\n..."


In [43]:
# lets clean up our index
client.ft(index_name).dropindex(True)

b'OK'

### What about JSON Support?

Redis also allows you to store data in JSON objects. The JSON fields can contain metadata and vectors. Below is a simple example of indexing JSON data.

In [44]:
# schema
schema = (
    TextField("$.content",                     # Text Field (JSON path)
        as_name="content"                      # Text Field Alias -- required for JSON
    ),
    VectorField("$.chunk_vector",              # Vector Field (JSON path)
        "FLAT", {                              # Vector Index Type: FLAT or HNSW
            "TYPE": "FLOAT32",
            "DIM": 384,                        # Number of Vector Dimensions
            "DISTANCE_METRIC": "COSINE",       # Vector Search Distance Metric
        },
        as_name="chunk_vector"                 # Vector Field Alias -- required for JSON
    ),
)

# index Definition
definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.JSON) # select JSON here

# create Index
client.ft(index_name).create_index(fields=schema, definition=definition)

b'OK'

In [45]:
client.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'JSON',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'$.content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'$.chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   384,
   b'distance_metric',
   b'COSINE']],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.00818634033203125',
 'total_inverted_index_blocks': '0',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': 'nan',
 'bytes_per_record_avg': 'nan',
 'offsets_per_term_avg': 'nan',
 'offset_bits_per_record_avg': 'nan',
 'hash_indexing_failures': '0',

In [46]:
# Write JSON data to the index

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        redis_key = f"{key_prefix}:{i}"
        data = {
            'content': chunk.page_content,
            'chunk_vector': chunk_embeddings[i].tolist() # notice that we don't need to convert JSON embeddings to bytes
        }
        #print(data)
        pipe.json().set(redis_key, "$", data)
        # mini batch
        if i % batch_size == 0:
            res = pipe.execute()

    res = pipe.execute() # make sure to use mini batches if working with larger datasets

In [47]:
# Fetch the JSON doc
client.json().get(f"{key_prefix}:0", "$")

[{'content': 'Table of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Mark One)☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE FISCAL YEAR ENDED MAY 31, 2023OR☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE TRANSITION PERIOD FROM TO .Commission File No. 1-10635',
  'chunk_vector': [-0.053405530750751495,
   -0.02921981550753117,
   -0.004023824818432331,
   0.05767367780208587,
   0.005367166828364134,
   0.06911122053861618,
   -0.008627132512629032,
   0.007825973443686962,
   -0.056115854531526566,
   0.03034288063645363,
   0.06386531889438629,
   -0.009552177973091602,
   -0.02332531102001667,
   -0.004573375452309847,
   -0.02859368547797203,
   0.03313298895955086,
   0.018091758713126183,
   0.03295215219259262,
   0.04179837554693222,
   0.07057030498981476,
   0.031152181327342987,
   -0.03115582838654518,
   -0.07718977332115173,
   -0.08126

In [48]:
# And now you can perform the same kinds of queries
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)

}
res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,content
0,doc:redispy:81,0.363789737225,"NIKE Brand revenues, which represented over 90..."
1,doc:redispy:79,0.616192996502,"(Dollars in millions, except per share data)\n..."


## Cleanup
Clean up the index and data.

In [49]:
client.ft(index_name).dropindex(True)

b'OK'

## What's Next?

Now that you have the basics down with the baseline Redis Python client:

| Notebook | Description | Documentation |
|----------|-------------|---------------|
| [**redisvl-02**](https://colab.research.google.com/github/redis-developer/financial-vss/blob/main/redisvl-02.ipynb) | Dive deeper using a dedicated VSS client library to build a RAG application from scratch including some advanced techniques using an OpenAI LLM. | [View Docs](https://redisvl.com) |
| [**langchain-03**](https://colab.research.google.com/github/redis-developer/financial-vss/blob/main/langchain-03.ipynb) | Wrap up with an integrated approach via LangChain and see how to build complex LLM chains. | [View Docs](https://python.langchain.com/docs/get_started/introduction) |
